In [ ]:

import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob
from scipy import stats

thedir='/Users/cslage/Research/LSST/optical_simulator/CTE/binning'
%cd $thedir

#%matplotlib inline

In [ ]:
files = sort(glob.glob('ITL-3800C-029_spot_spot_4??_20161121??????.fits'))
print files
name = files[0][24:27]
print name

In [ ]:
xstop = 64
ystop = 250
cti_list = []
skip_segments = ["SEGMENT00","SEGMENT01","SEGMENT07","SEGMENT10","SEGMENT17"]
for n,file in enumerate(files[0:5]):
    print file
    name = file[24:27]
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    exptime = hdulist[0].header['EXPTIME']
    fig = figure(figsize=(15,15))
    subplots_adjust(hspace = 0.3, wspace = 0.3)
    suptitle("Binned CTE(8x8) - Image %s, EXPTIME = %s seconds"%(name,exptime),fontsize=32)
    for segment in range(16):
        nmax = 0
        pointlist = []
        xaxis = linspace(-2,2,5)
        subplot(4,4,16-segment)
        imhdr=hdulist[segment+1].header
        extname = imhdr['EXTNAME']
        title("%s"%extname, fontsize=18)
        data = array(hdulist[segment+1].data + 32768, dtype = int32)[0:ystop,0:xstop]
        flat_overscan = median(data[0:ystop,0:xstop], axis = 1)
        cte_data = transpose(transpose(data) - flat_overscan)
        data_max = min(35000.0, 5000.0 * (int(name)-400+1))#cte_data.max()
        for i in range(2,xstop-2):
                for j in range(2,ystop-2):
                    # First keep only pixels aith near the total amount of charge
                    if cte_data[j,i] < data_max * 0.7:
                        continue
                    sum_surround = float(sum(cte_data[j-1:j+2,i-1:i+2]) - cte_data[j,i])
                    sum_2x = float(cte_data[j,i-2] + cte_data[j,i+2])
                    # Then keep only pixels where the surroundings have less than 2% of the center pixel
                    if (sum_surround / cte_data[j,i]) > 0.02 or (sum_2x / cte_data[j,i]) > 0.02:
                        continue
                    pointlist.append([i,j]) # this is a list of "good pixel locations
                    nmax += 1
        spotdata = zeros([nmax,5])
        cti_ratio = zeros([nmax])
        for m,[i,j] in enumerate(pointlist):
            spotdata[m,:] = cte_data[j,i-2:i+3] - (amin(cte_data[j,i-2:i+3]) - 1)
            cti_ratio[m] = float(cte_data[j,i+1] - cte_data[j,i-1]) / float(cte_data[j,i])
            if extname not in skip_segments:
                cti_list.append([i*8,cti_ratio[m],segment,float(exptime)])
        cti_mean = mean(cti_ratio)
        cti_std = std(cti_ratio)
        scatter(xaxis,mean(spotdata[:,:],axis=0))
        errorbar(xaxis, mean(spotdata[:,:],axis=0), yerr=std(spotdata[:,:],axis=0))
        plot([-2,2],[35000,35000], color = 'red', ls = '--')
        yscale('log')
        ylim(1.0,1.0E5)
        xlim(-2.2,2.2)
        text(-1.5,4.0E4,"Ratio = %.4f +/- %.4f"%(cti_mean,cti_std))
        print "%s, nmax = %d, cti_mean = %f"%(extname,nmax, cti_mean)
    savefig("Serial_CTE_Binning_%s_22Nov16.pdf"%name)
    close(fig)
figure()
title("CTI Ratios vs X-Position and Flux - 8x8")
for [x,ratio,segment,exptime] in cti_list: 
    if exptime < 0.6:
        scatter([x],[ratio],s=0.5,color = 'red')
        text(400,0.045,"exptime=0.5",color='red')
    elif exptime < 1.1:
        scatter([x+1.6],[ratio],s=0.5,color = 'blue')
        text(400,0.0425,"exptime=1.0",color='blue')
    elif exptime < 1.6:
        scatter([x+3.2],[ratio],s=0.5,color = 'green')
        text(400,0.040,"exptime=1.5",color='green')
    elif exptime < 2.1:
        scatter([x+4.8],[ratio],s=0.5,color = 'cyan')
        text(400,0.0375,"exptime=2.0",color='cyan')
    elif exptime < 2.6:
        scatter([x+6.4],[ratio],s=0.5,color = 'magenta')
        text(400,0.035,"exptime=2.5",color='magenta')
plot([0,520],[0.0,0.0], ls = '--', color = 'black')
ylim(-0.02, 0.05)
xlim(0,520)
xlabel("X Location")
ylabel("CTI Ratio")
savefig("Serial_CTE_Binning_Ratios_8x8_22Nov16.pdf")

